In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
#export
from exp.nb_11 import *

## Serializing the model

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=2920)

In [3]:
path = datasets.untar_data(datasets.URLs.IMAGEWOOF_160); path

WindowsPath('C:/Users/yago_/.fastai/data/imagewoof-160')

In [4]:
size = 128
bs = 64

tfms = [make_rgb, RandomResizedCrop(size, scale=(0.35,1)), np_to_float, PilRandomFlip()]
val_tfms = [make_rgb, CenterCrop(size), np_to_float]
il = ImageList.from_files(path, tfms=tfms)
sd = SplitData.split_by_func(il, partial(grandparent_splitter, valid_name='val'))
ll = label_by_func(sd, parent_labeler, proc_y=CategoryProcessor())
ll.valid.x.tfms = val_tfms
data = ll.to_databunch(bs, c_in=3, c_out=10, num_workers=1)

In [5]:
len(il)

12954

In [6]:
loss_func = LabelSmoothingCrossEntropy()
opt_func = adam_opt(mom=0.9, mom_sqr=0.99, eps=1e-6, wd=1e-2)

In [7]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [8]:
def sched_1cycle(lr, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [9]:
lr = 3e-3
pct_start = 0.5
cbsched = sched_1cycle(lr, pct_start)

In [10]:
learn.fit(40, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.103909,0.258150,2.057447,0.300000,00:45
1,1.926462,0.354183,1.990916,0.328000,00:41
2,1.786711,0.427011,1.788439,0.388000,00:41
3,1.709638,0.455195,1.788444,0.404000,00:42
4,1.634282,0.498233,1.859240,0.394000,00:42
5,1.598907,0.513731,2.065359,0.346000,00:41
6,1.549937,0.534206,1.689281,0.476000,00:43
7,1.506309,0.558054,1.651166,0.484000,00:43
8,1.454658,0.580456,1.570769,0.514000,00:43
9,1.403574,0.609282,1.699054,0.484000,00:44


In [11]:
st = learn.model.state_dict()

In [12]:
type(st)

collections.OrderedDict

In [13]:
', '.join(st.keys())

'0.0.weight, 0.1.weight, 0.1.bias, 0.1.running_mean, 0.1.running_var, 0.1.num_batches_tracked, 1.0.weight, 1.1.weight, 1.1.bias, 1.1.running_mean, 1.1.running_var, 1.1.num_batches_tracked, 2.0.weight, 2.1.weight, 2.1.bias, 2.1.running_mean, 2.1.running_var, 2.1.num_batches_tracked, 4.0.convs.0.0.weight, 4.0.convs.0.1.weight, 4.0.convs.0.1.bias, 4.0.convs.0.1.running_mean, 4.0.convs.0.1.running_var, 4.0.convs.0.1.num_batches_tracked, 4.0.convs.1.0.weight, 4.0.convs.1.1.weight, 4.0.convs.1.1.bias, 4.0.convs.1.1.running_mean, 4.0.convs.1.1.running_var, 4.0.convs.1.1.num_batches_tracked, 4.1.convs.0.0.weight, 4.1.convs.0.1.weight, 4.1.convs.0.1.bias, 4.1.convs.0.1.running_mean, 4.1.convs.0.1.running_var, 4.1.convs.0.1.num_batches_tracked, 4.1.convs.1.0.weight, 4.1.convs.1.1.weight, 4.1.convs.1.1.bias, 4.1.convs.1.1.running_mean, 4.1.convs.1.1.running_var, 4.1.convs.1.1.num_batches_tracked, 5.0.convs.0.0.weight, 5.0.convs.0.1.weight, 5.0.convs.0.1.bias, 5.0.convs.0.1.running_mean, 5.0.convs

In [14]:
st['10.bias']

tensor([ 0.0002,  0.0209, -0.0019, -0.0413, -0.0069, -0.0076, -0.0286, -0.0085,
         0.0490,  0.0249], device='cuda:0')

In [15]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)

It's also possible to save the whole model, including the architecture, but it gets quite fiddly and we don't recommend it. Instead, just save the parameters, and recreate the model directly.

In [16]:
torch.save(st, mdl_path/'iw5')

## Pets

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3127)

In [17]:
pets = datasets.untar_data(datasets.URLs.PETS)

In [18]:
pets.ls()

[WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/annotations'),
 WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images')]

In [19]:
pets_path = pets/'images'

In [20]:
il = ImageList.from_files(pets_path, tfms=tfms)

In [21]:
il

ImageList (7390 items)
[WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_1.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_10.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_100.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_101.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_102.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_103.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_104.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_105.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_106.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_107.jpg')...]
Path: C:\Users\yago_\.fastai\data\oxford-iiit-pet\images

In [22]:
#export
def random_splitter(fn, p_valid): return random.random() < p_valid

In [23]:
random.seed(42)

In [24]:
sd = SplitData.split_by_func(il, partial(random_splitter, p_valid=0.1))

In [25]:
sd

SplitData
Train: ImageList (6667 items)
[WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_1.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_100.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_101.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_102.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_103.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_104.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_106.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_108.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_109.jpg'), WindowsPath('C:/Users/yago_/.fastai/data/oxford-iiit-pet/images/Abyssinian_110.jpg')...]
Path: C:\Users\yago_\.fastai\data\oxford-iiit-pet\images
Valid: ImageList (723 items)
[WindowsPath

In [26]:
n = il.items[0].name; n

'Abyssinian_1.jpg'

In [27]:
re.findall(r'^(.*)_\d+.jpg$', n)[0]

'Abyssinian'

In [28]:
def pet_labeler(fn): return re.findall(r'^(.*)_\d+.jpg$', fn.name)[0]

In [29]:
proc = CategoryProcessor()

In [30]:
ll = label_by_func(sd, pet_labeler, proc_y=proc)

In [31]:
', '.join(proc.vocab)

'Abyssinian, american_bulldog, american_pit_bull_terrier, basset_hound, beagle, Bengal, Birman, Bombay, boxer, British_Shorthair, chihuahua, Egyptian_Mau, english_cocker_spaniel, english_setter, german_shorthaired, great_pyrenees, havanese, japanese_chin, keeshond, leonberger, Maine_Coon, miniature_pinscher, newfoundland, Persian, pomeranian, pug, Ragdoll, Russian_Blue, saint_bernard, samoyed, scottish_terrier, shiba_inu, Siamese, Sphynx, staffordshire_bull_terrier, wheaten_terrier, yorkshire_terrier'

In [32]:
ll.valid.x.tfms = val_tfms

In [33]:
c_out = len(proc.vocab)

In [34]:
data = ll.to_databunch(bs, c_in=3, c_out=c_out, num_workers=1)

In [35]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, norm=norm_imagenette)

In [36]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,3.450774,0.096145,3.422100,0.120332,01:35
1,3.224790,0.148793,4.347375,0.107884,00:56
2,2.974709,0.225739,3.382477,0.142462,00:54
3,2.651455,0.301335,2.623996,0.320885,00:51
4,2.337254,0.413979,2.367581,0.409405,00:51


## Custom head

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3265)

In [37]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [38]:
st = torch.load(mdl_path/'iw5')

In [39]:
m = learn.model

In [40]:
m.load_state_dict(st)

<All keys matched successfully>

We have to remove the last layer and introduce a new one. So we need to know the number of outputs for the previous.

In [41]:
cut = next(i for i,o in enumerate(m.children()) if isinstance(o,nn.AdaptiveAvgPool2d))
m_cut = m[:cut]

In [42]:
xb,yb = get_batch(data.valid_dl, learn)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [43]:
pred = m_cut(xb)

In [44]:
pred.shape

torch.Size([128, 512, 4, 4])

In [45]:
ni = pred.shape[1]

In [46]:
#export
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, sz=1):
        super().__init__()
        self.output_size = sz
        self.ap = nn.AdaptiveAvgPool2d(sz)
        self.mp = nn.AdaptiveMaxPool2d(sz)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)

In [47]:
nh = 40

m_new = nn.Sequential(
    m_cut, AdaptiveConcatPool2d(), Flatten(),
    nn.Linear(ni*2, data.c_out))

In [48]:
learn.model = m_new

In [49]:
learn.fit(5, cbsched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.855489,0.294885,2.166826,0.479945,00:54
1,1.944409,0.555572,2.144513,0.502075,00:59
2,1.715083,0.629669,1.883360,0.586445,00:56
3,1.487125,0.715914,1.517223,0.704011,00:56
4,1.283606,0.802910,1.429212,0.739972,00:57


## adapt_model and gradual unfreezing

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3483)

In [50]:
def adapt_model(learn, data):
    cut = next(i for i,o in enumerate(learn.model.children())
               if isinstance(o,nn.AdaptiveAvgPool2d))
    m_cut = learn.model[:cut]
    xb,yb = get_batch(data.valid_dl, learn)
    pred = m_cut(xb)
    ni = pred.shape[1]
    m_new = nn.Sequential(
        m_cut, AdaptiveConcatPool2d(), Flatten(),
        nn.Linear(ni*2, data.c_out))
    learn.model = m_new

In [51]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))

<All keys matched successfully>

In [52]:
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [53]:
for p in learn.model[0].parameters(): p.requires_grad_(False)

In [54]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.737799,0.308685,2.588994,0.380360,00:56
1,2.382728,0.438878,2.345768,0.439834,00:56
2,2.029451,0.533223,2.077747,0.521438,00:55


In [55]:
for p in learn.model[0].parameters(): p.requires_grad_(True)

In [56]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.825088,0.602520,1.873762,0.582296,00:56
1,1.714336,0.647668,1.920341,0.568465,00:55
2,1.627768,0.672416,1.745939,0.637621,00:55
3,1.435506,0.753562,1.575089,0.708160,00:55
4,1.276233,0.815959,1.474379,0.737206,00:57


It's worst than when we didn't any freezing. The problem is related to batch norm, and the trick to amend that is to not to freeze all the parameters in the net, but only the parameters not in the batch norm layers...

## Batch norm transfer

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3567)

In [57]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [58]:
def apply_mod(m, f):
    f(m)
    for l in m.children(): apply_mod(l, f)

def set_grad(m, b):
    if isinstance(m, (nn.Linear,nn.BatchNorm2d)): return
    if hasattr(m, 'weight'):
        for p in m.parameters(): p.requires_grad_(b)

In [59]:
apply_mod(learn.model, partial(set_grad, b=False))

In [60]:
learn.fit(3, sched_1cycle(1e-2, 0.5))

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.640585,0.333583,2.296659,0.442600,00:56
1,2.076095,0.503225,2.029454,0.536653,00:57
2,1.799669,0.615569,1.848074,0.603043,00:57


In [61]:
apply_mod(learn.model, partial(set_grad, b=True))

In [62]:
learn.fit(5, cbsched, reset_opt=True)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.699892,0.648118,1.817671,0.616874,00:56
1,1.661866,0.668367,2.036435,0.502075,00:58
2,1.595289,0.677966,1.783252,0.622407,00:57
3,1.426233,0.751912,1.521065,0.716459,00:59
4,1.261585,0.821809,1.447279,0.749654,00:58


Pytorch already has an `apply` method we can use:

In [63]:
learn.model.apply(partial(set_grad, b=False));

## Discriminative LR and param groups

[Jump_to lesson 12 video](https://course.fast.ai/videos/?lesson=12&t=3799)

In [64]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func, c_out=10, norm=norm_imagenette)

In [65]:
learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


We split the layers into two groups (batch norm layers and not):

In [66]:
def bn_splitter(m):
    def _bn_splitter(l, g1, g2):
        if isinstance(l, nn.BatchNorm2d): g2 += l.parameters()
        elif hasattr(l, 'weight'): g1 += l.parameters()
        for ll in l.children(): _bn_splitter(ll, g1, g2)
        
    g1,g2 = [],[]
    _bn_splitter(m[0], g1, g2)
    
    g2 += m[1:].parameters()
    return g1,g2

In [67]:
a,b = bn_splitter(learn.model)

In [68]:
test_eq(len(a)+len(b), len(list(m.parameters())))

In [69]:
Learner.ALL_CBS

{'after_backward',
 'after_batch',
 'after_cancel_batch',
 'after_cancel_epoch',
 'after_cancel_train',
 'after_epoch',
 'after_fit',
 'after_loss',
 'after_pred',
 'after_step',
 'begin_batch',
 'begin_epoch',
 'begin_fit',
 'begin_validate'}

In [70]:
#export
from types import SimpleNamespace
cb_types = SimpleNamespace(**{o:o for o in Learner.ALL_CBS})

In [71]:
cb_types.after_backward

'after_backward'

In [72]:
#export
class DebugCallback(Callback):
    _order = 999
    def __init__(self, cb_name, f=None): self.cb_name,self.f = cb_name,f
    def __call__(self, cb_name):
        if cb_name==self.cb_name:
            if self.f: self.f(self.run)
            else:      set_trace()

In [73]:
#export
def sched_1cycle(lrs, pct_start=0.3, mom_start=0.95, mom_mid=0.85, mom_end=0.95):
    phases = create_phases(pct_start)
    sched_lr  = [combine_scheds(phases, cos_1cycle_anneal(lr/10., lr, lr/1e5))
                 for lr in lrs]
    sched_mom = combine_scheds(phases, cos_1cycle_anneal(mom_start, mom_mid, mom_end))
    return [ParamScheduler('lr', sched_lr),
            ParamScheduler('mom', sched_mom)]

In [74]:
disc_lr_sched = sched_1cycle([0,3e-2], 0.5)

In [75]:
learn = cnn_learner(xresnet18, data, loss_func, opt_func,
                    c_out=10, norm=norm_imagenette, splitter=bn_splitter)

learn.model.load_state_dict(torch.load(mdl_path/'iw5'))
adapt_model(learn, data)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


In [76]:
def _print_det(o): 
    print (len(o.opt.param_groups), o.opt.hypers)
    raise CancelTrainException()

learn.fit(1, disc_lr_sched + [DebugCallback(cb_types.after_batch, _print_det)])

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time


2 [{'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0, 'sqr_mom': 0.99}, {'mom': 0.9499999999999997, 'mom_sqr': 0.99, 'eps': 1e-06, 'wd': 0.01, 'lr': 0.0030000000000000512, 'sqr_mom': 0.99}]


In [77]:
learn.fit(3, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,2.474325,0.381431,2.316417,0.449516,00:58
1,2.203220,0.461827,2.180043,0.475795,00:58
2,1.930470,0.554672,1.868825,0.605809,00:57


In [78]:
disc_lr_sched = sched_1cycle([1e-3,1e-2], 0.3)

In [79]:
learn.fit(5, disc_lr_sched)

epoch,train_loss,train_accuracy,valid_loss,valid_accuracy,time
0,1.755678,0.621269,1.860453,0.591978,01:01
1,1.778260,0.611219,2.001507,0.535270,00:59
2,1.627192,0.667917,1.750213,0.632089,00:58
3,1.509151,0.716364,1.640510,0.666667,01:00
4,1.402485,0.761512,1.599326,0.676349,01:00


## Export

In [80]:
!python notebook2script.py 11a_transfer_learning.ipynb

Converted 11a_transfer_learning.ipynb to exp\nb_11a.py
